#**Описание проекта**
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:

 - В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
 - Строят модель для предсказания объёма запасов в новых скважинах;
 - Выбирают скважины с самыми высокими оценками значений;
 - Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой _Bootstrap_.

#**Инструкция по выполнению проекта**
1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Обучите и проверьте модель для каждого региона:
 - 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
 - 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
 - 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
 - 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
 - 2.5. Проанализируйте результаты.
3. Подготовьтесь к расчёту прибыли:
 - 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
 - 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
 - 3.3. Напишите выводы по этапу подготовки расчёта прибыли.
4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
 - 4.1. Выберите скважины с максимальными значениями предсказаний.
 - 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
 - 4.3. Рассчитайте прибыль для полученного объёма сырья.
5. Посчитайте риски и прибыль для каждого региона:
 - 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
 - 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

**Описание данных**
Данные геологоразведки трёх регионов находятся в файлах:
 - /datasets/geo_data_0.csv. Скачать датасет
 - /datasets/geo_data_1.csv. Скачать датасет
 - /datasets/geo_data_2.csv. Скачать датасет
 - `id` — уникальный идентификатор скважины;
 - `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
 - `product` — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**

 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
 - Бюджет на разработку скважин в регионе — 10 млрд рублей.
 - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
 - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

##**1. Загрузите и подготовьте данные. Поясните порядок действий.**

In [217]:
def download_google_drive(link):
  url = link
  path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
  return pd.read_csv(path)

link_list = ['https://drive.google.com/file/d/1NLlZdInHKnE8KwIruEj6DeGbo4XGI_W0/view?usp=drive_link',
             'https://drive.google.com/file/d/15b8gAx9AU469qvgmYVUqxWZaRXzQVsk-/view?usp=drive_link',
             'https://drive.google.com/file/d/105txH8iCf_mqPJzwPP4TwLu1hXVCwgXj/view?usp=drive_link']

data_0, data_1, data_2 = [download_google_drive(link) for link in link_list]
list_df = [data_0, data_1, data_2]
data_0.name, data_1.name, data_2.name = ['data_0', 'data_1', 'data_2']


In [218]:
def df_info(df):
  print()
  print()
  print('________________________________________')
  print(f'Первые пять строк датафрейма {df.name}')
  display(df.head())
  print(f'Статистическая информация датафрейма {df.name}')
  display(df.describe())
  print(f'Общая информация о датафрейме {df.name}')
  print(df.info())
  print(f'Количество дублированных строк датафрема {df.name}: {df.duplicated().sum()}')

In [219]:
for df in list_df:
  df_info(df)



________________________________________
Первые пять строк датафрейма data_0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Статистическая информация датафрейма data_0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


Общая информация о датафрейме data_0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Количество дублированных строк датафрема data_0: 0


________________________________________
Первые пять строк датафрейма data_1


...

Общая информация о датафрейме data_1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Количество дублированных строк датафрема data_1: 0


________________________________________
Первые пять строк датафрейма data_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Статистическая информация датафрейма data_2


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Общая информация о датафрейме data_2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
Количество дублированных строк датафрема data_2: 0


In [220]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [221]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [222]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Выводы по первому пункту**
 - во всех трех датасетах отсутсвуют незаполненные значения и дублирующие строки, поэтому с ними можно работать;
 - столбец `id` не несет информационной составляющей для модели, поэтому удалим этот столбец из данных предварительно сохранив информацию о нем в словаре.

In [223]:
d0, d1, d2 = [df['id'].to_dict() for df in list_df]

In [224]:
data_0, data_1, data_2 = [df.drop('id', axis=1) for df in list_df]

##**2. Обучите и проверьте модель для каждого региона:**


###**2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.**

In [225]:
features_0 , target_0 = (data_0[['f0', 'f1', 'f2']], data_0['product'])
features_1 , target_1 = (data_1[['f0', 'f1', 'f2']], data_1['product'])
features_2 , target_2 = (data_2[['f0', 'f1', 'f2']], data_2['product'])


features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=.25, random_state=12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=.25, random_state=12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=.25, random_state=12345)

list_features_train = [features_train_0, features_train_1, features_train_2]
list_features_valid = [features_valid_0, features_valid_1, features_valid_2]
list_target_valid = [target_valid_0, target_valid_1, target_valid_2]

print(f'Размер обучающей выборки: {features_train_0.shape}')
print(f'Размер валидационной выборки: {features_valid_0.shape}')

Размер обучающей выборки: (75000, 3)
Размер валидационной выборки: (25000, 3)


In [226]:
#def scale(features_train, features_valid):
#    scaler = StandardScaler()
#    scaler.fit(features_train)
#    features_valid = pd.DataFrame(scaler.transform(features_valid),
#                                columns = features_valid.columns)
#    return features_valid

In [227]:
#features_valid_0, features_valid_1, features_valid_2 = [
#    scale(f_train, f_valid)
#    for f_train, f_valid in zip(list_features_train, list_features_valid)]

###**2.2. Обучите модель и сделайте предсказания на валидационной выборке.**

In [228]:
def get_predicted_mean_rmse(features_train, target_train, features_valid, target_valid):
  model = LinearRegression()
  model.fit(features_train, target_train)
  predicted_valid = model.predict(features_valid)
  predicted_valid = pd.Series(predicted_valid, index=target_valid.index)
  rmse = root_mean_squared_error(target_valid, predicted_valid)
  return predicted_valid.mean(), rmse, predicted_valid

###**2.3. Сохраните предсказания и правильные ответы на валидационной выборке.**

In [229]:
list_features_targets = [[features_train_0, target_train_0, features_valid_0, target_valid_0],
                         [features_train_1, target_train_1, features_valid_1, target_valid_1],
                         [features_train_2, target_train_2, features_valid_2, target_valid_2]]

list_mean = []
list_rmse = []
list_predictions = []

for row in  list_features_targets:
  list_mean.append(get_predicted_mean_rmse(*row)[0])
  list_rmse.append(get_predicted_mean_rmse(*row)[1])
  list_predictions.append(get_predicted_mean_rmse(*row)[2])
d = {}
d['Cредний запас предсказанного сырья, тыс. баррелей'] = list_mean
d['RMSE модели, тыс. баррелей'] = list_rmse
result_df = pd.DataFrame(data=d, index=['Регион_0', 'Регион_1', 'Регион_2'])






###**2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.**

In [230]:
result_df

,"Cредний запас предсказанного сырья, тыс. баррелей","RMSE модели, тыс. баррелей"
Регион_0,92.592568,37.579422
Регион_1,68.728547,0.893099
Регион_2,94.965046,40.029709


###**2.5. Проанализируйте результаты.**

Регионы с индексом `0` и `2` имеют приблизительно одинаковые характеристики по предсказанным средним запасам и RMSE модели. Регион `1` имеет меньший предсказанный запас сырья, но RMSE = 0.89, это говорит о том, что модель для региона `1` гораздо меньше ошибается в своих предсказаниях.

##**3. Подготовьтесь к расчёту прибыли:**

###**3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.**

In [231]:
BOOTSTRAP_NUMBER_SIZE = 1000        # количество итераций для техники Bootstrap
BOOTSTRAP_SIZE_SAMPLE = 500         # количество точек для иссследования в каждом регионе
BOOTSTRAP_BEST_RESULT_SIZE = 200    # количество точек с лучшими параметрами
REVENUE_THOUSAND_BARRELS = 450000   # оборот за каждую тысячу баррелей
REGION_BUDGET = 10**10              # бюджет на один регион

###**3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.**

In [232]:
value_sufficient = REGION_BUDGET/(REVENUE_THOUSAND_BARRELS*BOOTSTRAP_BEST_RESULT_SIZE)
print(f'Достаточный объем сырья для безубыточной разработки: {value_sufficient:.2f} тыс. баррелей')

Достаточный объем сырья для безубыточной разработки: 111.11 тыс. баррелей


Средние предсказанные запасы для регионов 92.59, 68.72, 94.96 тыс. баррелей соответсвенно. Для безубыточной добычи каждая скважина должна приносить в среднем 111.11 тыс. баррелей.

###**3.3. Напишите выводы по этапу подготовки расчёта прибыли.**

##**4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:**
 - 4.1. Выберите скважины с максимальными значениями предсказаний.
 - 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
 - 4.3. Рассчитайте прибыль для полученного объёма сырья.

In [233]:
def profit(target, predictions, count):
  predictions = predictions.sort_values(ascending=False)
  ind = predictions.index[:count]
  p = (target[ind]).sum()
  return p*REVENUE_THOUSAND_BARRELS - REGION_BUDGET

##**5. Посчитайте риски и прибыль для каждого региона:**
 - 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.


In [234]:
def calculate_profit(target, predictions):
  predictions = pd.Series(predictions)
  state = np.random.RandomState(12345)
  values = []
  counter = 0
  for i in range(BOOTSTRAP_NUMBER_SIZE):
    sub_sample = predictions.sample(BOOTSTRAP_SIZE_SAMPLE, replace=True, random_state=state)
    values.append(profit(target[sub_sample.index], sub_sample, BOOTSTRAP_BEST_RESULT_SIZE))
    if values[i] < 0:
      counter +=1

  values = pd.Series(values)
  up = values.quantile(0.025)
  down = values.quantile(0.975)
  return round(values.mean()/10**9,3), (counter*100)/BOOTSTRAP_NUMBER_SIZE, (
      '[' + str(round(up/10**9, 3)) + '; ' + str(round(down/10**9, 3)) + ']')

In [235]:
result = []
for t, p in zip(list_target_valid, list_predictions):
  result.append(calculate_profit(t,p))

In [236]:
result

[(0.601, 2.0, '[0.013; 1.231]'),
 (0.665, 0.3, '[0.158; 1.198]'),
 (0.616, 3.0, '[-0.012; 1.231]')]

In [237]:
result_df = pd.DataFrame(data=result,
            index = ['Регион_0', 'Регион_1', 'Регион_2'],
            columns = ['Средняя прибыль, млрд. руб', 'Вероятность убытка, %', '95% доверительный интервал для прибыли, млрд. руб'])

In [238]:
result_df

,"Средняя прибыль, млрд. руб","Вероятность убытка, %","95% доверительный интервал для прибыли, млрд. руб"
Регион_0,0.601,2.0,[0.013; 1.231]
Регион_1,0.665,0.3,[0.158; 1.198]
Регион_2,0.616,3.0,[-0.012; 1.231]


 - 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

По условию задачи нужно остановиться на регионах, у которых  вероятность убытков меньше 2.5% и среди них выбрать регион с наибольшей средней прибылью. Этому условию удовлетворяют два региона: `0` и `1` c вероятностью убытков 2.0% и 0.3% соответсвенно. Среди этих регионов наибольшая средняя прибыль у региона `1`, так как у него средняя прибыль больше.
  Вывод: для разработки новой скважины стоит выбрать регион `1`